In [ ]:
# train_cnn.ipynb

In [6]:
# Imports

# Needed for grabbing local python files
import sys
import os

# ML library for creating models and training
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

# Used to iterate over a dataset
from torch.utils.data import random_split, DataLoader

# Show data
import matplotlib.pyplot as plt

In [11]:
# Grab local imports
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(project_root, "src"))

from data.load_data import simple_train_dl, simple_val_dl
from models.cnn import CNN
from utils.progress import progress

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [12]:
# Initialize model
cnn_model = CNN().to(device)

In [9]:
def train_and_evaluate(model, train_dataloader, test_dataloader, epochs=10, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = nn.SmoothL1Loss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_loss_history = []
    test_loss_history = []

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0

        for images, labels, _ in progress(train_dataloader, total_steps=len(train_dataloader)):
            images, labels = images.to(device), labels.to(device).float()
            #print('shp', images.shape)
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_dataloader)
        train_loss_history.append(avg_train_loss)

        # Evaluate on test data
        model.eval()
        total_test_loss = 0

        with torch.no_grad():
            for images, labels in test_dataloader:
                #print(images.squeeze().shape)
                images, labels = images.to(device), labels.to(device).float()
                outputs = model(images).squeeze()
                loss = criterion(outputs, labels)
                total_test_loss += loss.item()

        avg_test_loss = total_test_loss / len(test_dataloader)
        test_loss_history.append(avg_test_loss)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

    # Plot Learning Curve
    plt.plot(range(1, epochs+1), train_loss_history, label="Train Loss", marker='o')
    plt.plot(range(1, epochs+1), test_loss_history, label="Test Loss", marker='o')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Learning Curve")
    plt.legend()
    plt.grid(True)
    plt.show()

In [10]:
train_and_evaluate(cnn_model, simple_train_dl, simple_val_dl, epochs=10)

Progress: 100.00% completed | ETA: 00m 00s


ValueError: too many values to unpack (expected 2)

In [ ]:

def evaluate_model(model, test_dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    total_samples = 0
    correct_predictions = 0
    total_absolute_error = 0.0

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels, _ in test_dataloader:  # Assuming the third value is metadata (not used here)
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images).squeeze()

            predictions = torch.round(outputs)  # Assuming binary classification
            correct_predictions += (predictions == labels).sum().item()
            total_samples += labels.size(0)

            total_absolute_error += F.l1_loss(outputs, labels, reduction="sum").item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(outputs.cpu().numpy())

    accuracy = correct_predictions / total_samples * 100
    mean_absolute_error = total_absolute_error / total_samples

    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Mean Absolute Error: {mean_absolute_error:.4f}")

    return {
        "accuracy": accuracy,
        "mean_absolute_error": mean_absolute_error,
        "labels": all_labels,
        "predictions": all_predictions
    }
